# IPython Parallel with Singularity
---
Singularity containers can be used to execute ipengines on Summit, providing a mechanism for multi-node ipyparallel workflows that run in custom software environments.

## Pull the base image
Since bootstrapping from Singularity Hub is currently unsupported, we will instead pull the CURC base image from our Docker Hub organization _(let me know if you'd like to be added to it)_. This base image includes `ipyparallel==6.0.2`.

Before we pull, we will also change the directory Singularity uses for caching, to avoid filling up our home directory.

In [1]:
ml python/3.5.1 singularity/2.3.2
export SINGULARITY_CACHEDIR=/projects/$USER/.singularity
singularity pull --name "jupyter-ipyparallel.img" docker://researchcomputing/jupyter-base

## Make a new IPython parallel profile
Now that the image has been pulled, we need to make a new IPython parallel profile to work with. If you wish to start from scratch, run
```
ipython profile create --parallel --profile=singularity-shas
```

It is quickest to start from the example profile we provide in your home directory though:

In [3]:
cp -a ~/.ipython/profile_example-shas ~/.ipython/profile_singularity-shas
ls ~/.ipython/

extensions    profile_default	    profile_singularity-shas
nbextensions  profile_example-shas


# Edit the cluster config
We need to edit the cluster config _(located at `~/.ipython/profile_singularity-shas/ipcluster_config.py`)_ to use `singularity exec`.

```python
# Configuration file for ipcluster.

c.IPClusterEngines.engine_launcher_class = 'ipyparallel.apps.launcher.SlurmEngineSetLauncher'
c.SlurmLauncher.qos = 'normal'
c.SlurmLauncher.timelimit = '4:00:00'
# c.SlurmLauncher.account = ''
# c.SlurmLauncher.machines = ''
# c.SlurmLauncher.mem = ''
# c.SlurmLauncher.resources = ''

c.SlurmEngineSetLauncher.batch_template = """#!/bin/bash

#SBATCH --partition shas
#SBATCH --qos {qos}
#SBATCH --job-name singularity-ipengine
#SBATCH --ntasks {n}
# This will run a single ipengine per CPU
#SBATCH --cpus-per-task 1
# Use ntasks-per-node=1 to run one ipengine per node
#SBATCH --time {timelimit}
#SBATCH --output {profile_dir}/log/slurm.out

module load singularity/2.3.2 impi

# This line executes each ipengine in a Singularity container, and aggregates it into an MPI pool.
mpirun singularity exec -e /projects/sampedro/singularity-jupyter-demo/jupyter-rc.img ipengine --profile-dir="{profile_dir}" --cluster-id="{cluster_id}"
"""
```